In [2]:
import pandas as pd
##pd.set_option("display.max_rows", None)
import re
import numpy as np
import os
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
###importing the data and checking through for anormalies 
data2018 = pd.read_csv('2018b table uncleaned .csv', parse_dates=['date'])
data2018.tail(50)
data2018.isnull().sum()

viewid         0
date           0
pagePath       0
hits           0
dimension36    0
dtype: int64

In [4]:
##creating a check point and droping the first 4 rows##
data_to_clean = data2018[4:]
data_to_clean.columns

Index(['viewid', 'date', 'pagePath', 'hits', 'dimension36'], dtype='object')

creating new column to group new model from old model

In [5]:
#data_to_clean2= data2018[4:50]
#data_to_clean['pagePath']
#data_to_clean.head(20)


##creating function for new or old##
#newness = []
#new = 'new_car' 
#used = 'used_car'

#for i in data_to_clean['pagePath']:
#    if data_to_clean.pagePath[i].str.contains('new', regex=False):
#        newness.append[new]
#    else: 
 #       newness.append[used]
   

In [6]:
##creating function for to find out whether car is new or new or old (Newness Type) ##
data_to_clean["newness_type"]=data_to_clean["pagePath"].str.extract('(new|used)')
data_to_clean['newness_type'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type'].unique() ##check unique values 
data_to_clean['newness_type'].value_counts() ##count unique values 

<ipython-input-6-74534ebc6c11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean["newness_type"]=data_to_clean["pagePath"].str.extract('(new|used)')
/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


new             159619
used             40980
unknown_type      9318
Name: newness_type, dtype: int64

In [7]:
###viewing data without unknown newness type##
data_to_clean[data_to_clean['newness_type']== 'unknown_type'].head(20)

,viewid,date,pagePath,hits,dimension36,newness_type
335,129875374,2018-01-01,/mobile_lead_form_thank_you_carcode,2,https://www.edmunds.com/inventory/gp_thank_you...,unknown_type
351,129875374,2018-01-01,/cpo_model_car_inventory_form_price_promise_op...,40,https://www.edmunds.com/inventory/used/lead_fo...,unknown_type
353,129875374,2018-01-01,/mobile_cpo_model_car_inventory_form_price_pro...,16,https://www.edmunds.com/inventory/used/lead_fo...,unknown_type
709,129875374,2018-01-02,/vdp,2,https://statefarm.edmunds.com/details/?vin=2T1...,unknown_type
710,129875374,2018-01-02,/vdp,4,https://statefarm.edmunds.com/details/?vin=2T3...,unknown_type
711,129875374,2018-01-02,/srp,4,https://statefarm.edmunds.com/srp/?make=Volvo%...,unknown_type
1098,129875374,2018-01-02,/cpo_model_car_inventory_form_price_promise_op...,29,https://www.edmunds.com/inventory/used/lead_fo...,unknown_type
1100,129875374,2018-01-02,/mobile_cpo_model_car_inventory_form_price_pro...,23,https://www.edmunds.com/inventory/used/lead_fo...,unknown_type
1479,129875374,2018-01-03,/vdp,2,https://statefarm.edmunds.com/details/?vin=2HG...,unknown_type
1480,129875374,2018-01-03,/vdp,4,https://statefarm.edmunds.com/details/?vin=SHH...,unknown_type


In [8]:
data_to_clean['newness_type'].unique()
data_to_clean['newness_type'].value_counts()



new             159619
used             40980
unknown_type      9318
Name: newness_type, dtype: int64

In [9]:
##creating function for to find out whether car is new or new or old in Dimension36 (As Newness Type) ##
data_to_clean["newness_type2"]=data_to_clean["dimension36"].str.extract('(new|used)')
data_to_clean['newness_type2'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type2'].unique() ##check unique values 
data_to_clean['newness_type2'].value_counts() ##count unique values 

<ipython-input-9-5d422a88427f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean["newness_type2"]=data_to_clean["dimension36"].str.extract('(new|used)')
/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


unknown_type    114348
new              79867
used             15702
Name: newness_type2, dtype: int64

In [10]:
check = ["newness_type2","newness_type"]
data_to_clean[check]
#data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,newness_type2,newness_type
4,unknown_type,used
5,unknown_type,used
6,unknown_type,new
7,unknown_type,new
8,unknown_type,new
...,...,...
209916,unknown_type,new
209917,unknown_type,used
209918,unknown_type,used
209919,unknown_type,new


In [11]:
data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,viewid,date,pagePath,hits,dimension36,newness_type,newness_type2
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,unknown_type
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,unknown_type
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new,unknown_type
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new,unknown_type
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new,unknown_type
...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new,unknown_type
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown_type
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown_type
209919,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new,unknown_type


In [12]:
### merging btoh newness column to have one column## 
data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2

<ipython-input-12-162d5016c148>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2
/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [13]:
data_to_clean['newness_type'].value_counts()

new             165815
used             41771
unknown_type      2331
Name: newness_type, dtype: int64

In [14]:
data_to_clean[data_to_clean['newness_type']== 'unknown_type'].tail(100)

,viewid,date,pagePath,hits,dimension36,newness_type,newness_type2
191249,129875374,2018-12-30,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type
191250,129875374,2018-12-30,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type
191741,129875374,2018-12-31,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type
191742,129875374,2018-12-31,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type
191743,129875374,2018-12-31,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type
...,...,...,...,...,...,...,...
209217,129875374,2018-08-09,/car_inventory_ddp_map,2,https://www.edmunds.com/ford/f-150/2009/vin/1F...,unknown_type,unknown_type
209228,129875374,2018-08-09,/car_inventory_ddp_map,2,https://www.edmunds.com/ford/flex/2010/vin/2FM...,unknown_type,unknown_type
209530,129875374,2018-08-09,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type
209531,129875374,2018-08-09,/hyundai_test_drive_promo,2,https://www.edmunds.com/hyundai-test-drive-pro...,unknown_type,unknown_type


In [32]:
##clean_1 = data_to_clean[data_to_clean.newness_type!= 'unknown_type']
clean_1 = data_to_clean.drop(['newness_type2'], axis=1)##break point after removing newness_type2 data  or newness or use type

###clean_1.drop(['newness_type2'], axis=1, inplace=True) ##code if if I have to remove the unknown newness type  

In [33]:
clean_1.head() ##previewing data 

,viewid,date,pagePath,hits,dimension36,newness_type
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new


In [34]:
#####working on the url (dimension36) too extract importanst data#
###first removing the url beginning ###
#clean_1["dimension36"].replace(regex=['https://www.edmunds.com/'], value=[''], inplace = True) ##removing the beginning url for consistencies in the column 


In [35]:
clean_2= clean_1.copy()
clean_2

,viewid,date,pagePath,hits,dimension36,newness_type
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new
...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used
209919,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new


In [19]:
#clean_1[128317:128320]

In [36]:
clean_2.dimension36.dtypes

dtype('O')

In [59]:
####EXtracting Values for make, model and year
clean_2["Maker"]=clean_2["dimension36"].str.extract('(make=...............)')
clean_2["Model"]=clean_2["dimension36"].str.extract('(model=.............)')
clean_2["Year"]=clean_2["dimension36"].str.extract('(year=...............)')


In [68]:
clean_3= clean_1.copy()
clean_3

,viewid,date,pagePath,hits,dimension36,newness_type
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new
...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used
209919,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new


In [99]:
####EXtracting Values for make, model and year
clean_3["Maker"]=clean_3["dimension36"].str.extract('(make=...............)')
clean_3["Model"]=clean_3["dimension36"].str.extract('(model=.............)')
clean_3["Year"]=clean_3["dimension36"].str.extract('(year=...............)')




clean_3['Maker'].fillna('unknown', inplace=True)
clean_3['Model'].fillna('unknown', inplace=True)
clean_3['Year'].fillna('unknown', inplace=True)



clean_3["Make2"]=clean_3["Maker"].apply(lambda st: st[st.find("make=")+5:st.find("&")])
clean_3['Make2'].replace({"ow": "unknown"}, inplace=True)
                          

In [100]:
clean_3[clean_3['Maker']!='unknown'].tail()

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year,Make2
209827,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Lincoln&model=M,unknown,year=2016&make=Linco,Lincoln
209829,129875374,2018-08-09,/mobile_used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Nissan&model=Al,unknown,year=2016&make=Nissa,Nissan
209830,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Volkswagen&mode,unknown,year=2016&make=Volks,Volkswagen
209832,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Jeep&model=Cher,unknown,year=2017&make=Jeep&,Jeep
209833,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Ford&model=Fusi,unknown,year=2018&make=Ford&,Ford


In [101]:
clean_3[clean_3['Maker']=='unknown'].tail()

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year,Make2
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new,unknown,unknown,unknown,unknown
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown,unknown,unknown,unknown
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown,unknown,unknown,unknown
209919,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new,unknown,unknown,unknown,unknown
209920,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc90/2018/suv/,new,unknown,unknown,unknown,unknown


In [60]:
clean_2

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,NaN,NaN,NaN
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,NaN,NaN,NaN
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new,NaN,NaN,NaN
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new,NaN,NaN,NaN
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new,NaN,NaN,NaN
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,NaN,NaN,NaN
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,NaN,NaN,NaN
209919,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new,NaN,NaN,NaN


In [61]:
clean_2[clean_2.Maker.notnull()].head()

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year
486,129875374,2018-01-01,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=BMW&model=3%20S,NaN,year=2003&make=BMW&m
487,129875374,2018-01-01,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Lexus&model=ES,NaN,year=2003&make=Lexus
489,129875374,2018-01-01,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Dodge&model=Int,NaN,year=2004&make=Dodge
490,129875374,2018-01-01,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Chevrolet&model,NaN,year=2006&make=Chevr
491,129875374,2018-01-01,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Chevrolet&model,model=Monte%20Carlo,year=2006&make=Chevr


In [62]:
clean_2[clean_2.Maker.isnull()].head()
###clean_1['make2'].notnull().sum()   #####count with 

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,NaN,NaN,NaN
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,NaN,NaN,NaN
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new,NaN,NaN,NaN
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new,NaN,NaN,NaN
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new,NaN,NaN,NaN


In [63]:
###converting NaN to string ###
clean_2['Maker'].fillna('unknown', inplace=True)
clean_2['Model'].fillna('unknown', inplace=True)
clean_2['Year'].fillna('unknown', inplace=True)

In [64]:
clean_2[clean_2.Maker=='unknown'].head()
###clean_1['make2'].notnull().sum()   #####count with 

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,unknown,unknown,unknown
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,unknown,unknown,unknown
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new,unknown,unknown,unknown
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new,unknown,unknown,unknown
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new,unknown,unknown,unknown


In [54]:
clean_2.Make2[clean_2.Make2!='unknown'].value_counts()

ow                108197
honda              18857
toyota             12079
subaru              5325
ford                4139
                   ...  
Plymouth               1
lincoln%26mode         1
nissan%26model         1
kia%26model=Sp         1
ram%26model=25         1
Name: Make2, Length: 140, dtype: int64

In [65]:
#clean_2["Maker"]= clean_2["Maker"].apply(str)
#clean_1["OEM"]=clean_1["Make"].apply(lambda st: st[st.find("make=")+5:st.find("&")])
clean_2["Make2"]=clean_2["Maker"].apply(lambda st: st[st.find(("make="))+5:st.find(("&"))])

In [66]:
clean_2

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year,Make2
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,unknown,unknown,unknown,ow
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,unknown,unknown,unknown,ow
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new,unknown,unknown,unknown,ow
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new,unknown,unknown,unknown,ow
8,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2018/suv/#ed...,new,unknown,unknown,unknown,ow
...,...,...,...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new,unknown,unknown,unknown,ow
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown,unknown,unknown,ow
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown,unknown,unknown,ow
209919,129875374,2018-08-09,/new_model_core,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new,unknown,unknown,unknown,ow


In [67]:
clean_2[clean_2['Maker']!='unknown'].tail()

,viewid,date,pagePath,hits,dimension36,newness_type,Maker,Model,Year,Make2
209827,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Lincoln&model=M,unknown,year=2016&make=Linco,Lincoln
209829,129875374,2018-08-09,/mobile_used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Nissan&model=Al,unknown,year=2016&make=Nissa,Nissan
209830,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Volkswagen&mode,unknown,year=2016&make=Volks,Volkswagen
209832,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Jeep&model=Cher,unknown,year=2017&make=Jeep&,Jeep
209833,129875374,2018-08-09,/used_model_mydp_var_thank_you,2,https://www.edmunds.com/myappraise/thank-you.h...,used,make=Ford&model=Fusi,unknown,year=2018&make=Ford&,Ford


In [946]:
####clean_try["make"]=clean_try["dimension36"].apply(lambda st: st[st.find("make=")+5:st.find("&model")-2])### original code



In [947]:
clean_1["Make"]= clean_1["Make"].apply(str)
#clean_1["OEM"]=clean_1["Make"].apply(lambda st: st[st.find("make=")+5:st.find("&")])
clean_1["make2"]=clean_1["Make"].apply(lambda st: st[st.find("make=")+5:st.find("&model")])

#clean_1["Year_of_build"]=clean_1["Year"].apply(lambda st: st[st.find("year=")+5:st.find("&")])
#clean_1["model"]=clean_1["Model"].apply(lambda st: st[st.find("model=")+6:st.find("&")])

In [30]:
clean_1[clean_1['make2'].isnull()]

KeyError: 'make2'

In [806]:
#pattern = '/groundwork/lead/form/pricing/?buildData=false&callToAction=purchaseOffer&leadSource=core&'
#first_pattern ='groundwork/lead/form/pricing/\?buildData=false&callToAction=purchaseOffer&leadSource=core&'
#clean_try['dimension36'] = clean_try['dimension36'].str.replace((first_pattern), '',)

In [807]:

clean_try.dimension36

128321    groundwork/lead/form/pricing/?buildData=false&...
128322    groundwork/lead/form/pricing/?buildData=false&...
128323    groundwork/lead/form/pricing/?buildData=false&...
Name: dimension36, dtype: object

In [808]:
##clean_try['dimension36'] = clean_try['dimension36'].str.replace(('groundwork/lead/form/pricing/'), '')  ##one solution 

In [809]:
clean_try["make"]

128321    linco
128322    linco
128323    linco
Name: make, dtype: object

In [814]:
clean_1[86649:]

,viewid,date,pagePath,hits,dimension36,newness_type,Make,Model,Year,OEM,Year_of_build,model
86653,129875374,2018-09-19,/mobile_new_model_mydp_dl_thank_you,2,ford/flex/2018/thankyou.html;jsessionid=8DB54A...,new,NaN,NaN,NaN,NaN,NaN,NaN
86654,129875374,2018-09-19,/mobile_new_model_mydp_dl_thank_you,2,ford/fusion/2018/thankyou.html;jsessionid=C73C...,new,NaN,NaN,NaN,NaN,NaN,NaN
86655,129875374,2018-09-19,/mobile_new_model_mydp_dl_thank_you,2,ford/fusion/2019/thankyou.html?submit=true,new,NaN,NaN,NaN,NaN,NaN,NaN
86656,129875374,2018-09-19,/mobile_new_model_mydp_dl_thank_you,2,ford/mustang/2017/thankyou.html?submit=true,new,NaN,NaN,NaN,NaN,NaN,NaN
86657,129875374,2018-09-19,/mobile_new_model_mydp_dl_thank_you,2,ford/mustang/2018/thankyou.html;jsessionid=FE3...,new,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,volvo/xc40/2019/thankyou.html?submit=true,new,NaN,NaN,NaN,NaN,NaN,NaN
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,volvo/xc60/2017/vin/YV449MRR7H2064128/,used,NaN,NaN,NaN,NaN,NaN,NaN
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,volvo/xc60/2017/vin/YV449MRRXH2223420/,used,NaN,NaN,NaN,NaN,NaN,NaN
209919,129875374,2018-08-09,/new_model_core,2,volvo/xc60/2018/?utm_medium=sem&utm_source=goo...,new,NaN,NaN,NaN,NaN,NaN,NaN


In [793]:
clean_1[clean_1['model']!= 'NaN']

,viewid,date,pagePath,hits,dimension36,newness_type,Make,Model,Year,OEM,Year_of_build,model
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,Chevrolet/Tahoe/2015/vin/1GNSKBKC5FR180478/?re...,used,NaN,NaN,NaN,NaN,NaN,NaN
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,Ford/F-150/2012/vin/1FTFW1ET5CFB39172/?utm_sou...,used,NaN,NaN,NaN,NaN,NaN,NaN
6,129875374,2018-01-01,/new_model_core,2,acura/ilx/2017/sedan/#edm-entry-pricing,new,NaN,NaN,NaN,NaN,NaN,NaN
7,129875374,2018-01-01,/new_model_core,2,acura/mdx/2017/#edm-entry-pricing,new,NaN,NaN,NaN,NaN,NaN,NaN
8,129875374,2018-01-01,/new_model_core,2,acura/mdx/2018/suv/#edm-entry-pricing,new,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,/mobile_new_model_mydp_dl_thank_you,2,volvo/xc40/2019/thankyou.html?submit=true,new,NaN,NaN,NaN,NaN,NaN,NaN
209917,129875374,2018-08-09,/used_model_car_inventory_vin_detail,4,volvo/xc60/2017/vin/YV449MRR7H2064128/,used,NaN,NaN,NaN,NaN,NaN,NaN
209918,129875374,2018-08-09,/used_model_car_inventory_vin_detail,2,volvo/xc60/2017/vin/YV449MRRXH2223420/,used,NaN,NaN,NaN,NaN,NaN,NaN
209919,129875374,2018-08-09,/new_model_core,2,volvo/xc60/2018/?utm_medium=sem&utm_source=goo...,new,NaN,NaN,NaN,NaN,NaN,NaN


In [791]:
# import re

#text = 'gfgfdAAA1234ZZZuijjk'

#m = re.search('make=(.+?)&', clean_try["dimension36"]).group(1)
#if m:
#   clean_try["make2"] = m.group(1)

#/* [MCCOOK 0     ] */,999990,'MCCOOK  0   '
#I want to extract the substring between [ and ] into another column. Please advise.

#your_column.apply(lambda st: st[st.find("[")+1:st.find("]")])
# s.apply(lambda st: st[st.find("(")+1:st.find(")")])

clean_try["make"]=clean_try["dimension36"].apply(lambda st: st[st.find("make=")+5:st.find("&model")-2])